### 🟢 STEP-1 — Load old model + New retrained model

In [1]:
# load models and encoder
import joblib

old_model = joblib.load("beekin_model_baseline.pkl")     # previous version
new_model = joblib.load("beekin_model_2022.pkl")         # retrained model
enc = joblib.load("property_type_encoder.pkl")


###🟢 STEP-2 — Prepare evaluation dataset

In [3]:
# load same test dataset for fair comparison
import pandas as pd
test = pd.read_csv("test.csv")

# apply same encoding to property_type column using saved encoder
test["property_type_enc"] = enc.transform(test["property_type"])

# define baseline feature set (must match training features exactly)
baseline_features = [
    "latitude","longitude","sqft","beds","baths",
    "property_type_enc","days_since_2014"
]

target_col = "trans_log_price"

X_test = test[baseline_features]
y_test = test[target_col]

### 🟢 STEP-3 — Evaluate both models (detailed)

In [4]:
# evaluate both models on same test data
from numpy import median, abs as np_abs

# Define MdAPE — Median Absolute Percentage Error
def mdape(y_true, y_pred):
    return median(np_abs((y_true - y_pred) / y_true) * 100)

# generate predictions using new retrained model and old model
old_mdape = mdape(y_test, old_model.predict(X_test))
new_mdape = mdape(y_test, new_model.predict(X_test))

print("Old Model MdAPE :", old_mdape)
print("New Model MdAPE :", new_mdape)


Old Model MdAPE : 38.69357283150387
New Model MdAPE : 40.32409177498017


###🟢 STEP-4 — Apply monitoring rule (tolerance threshold)

In [5]:
# define tolerance threshold for performance degradation
TOLERANCE = 1.0   # 1% MdAPE change allowed

# compute difference between new and old performance
performance_change = new_mdape - old_mdape
print("Performance Change (+worse / -better):", performance_change)

# decision rule for acceptance or rollback
if performance_change > TOLERANCE:
    print("⚠️ Alert: New model is worse — recommend rollback")
else:
    print("✅ New model performance is acceptable")


Performance Change (+worse / -better): 1.6305189434762966
⚠️ Alert: New model is worse — recommend rollback


### 🟢 STEP-5 — Save monitoring report

In [6]:
import json

monitor_report = {
    "old_mdape": float(old_mdape),
    "new_mdape": float(new_mdape),
    "performance_change": float(performance_change),
    "status": "rollback" if performance_change > TOLERANCE else "accept"
}

with open("model_monitor_report.json", "w") as f:
    json.dump(monitor_report, f, indent=2)

print("Monitoring report saved")


Monitoring report saved
